In [4]:
import pandas as pd

import numpy as np

#Leitura do dataset - 50.000 textos rotulados em positivo e negativo

db_pos_ml = pd.read_csv(r'ml.pos',header=None, encoding="UTF-8")
db_neg_ml = pd.read_csv(r'ml.neg',header=None, encoding="UTF-8")

#db_pos_buscape = pd.read_csv('E:/Pos UFG/Disciplinas/Analise_de_Sentimentos/Trabalho3/buscape1.pos',header=None, encoding="ISO-8859-1")

db_pos_ml = np.array(db_pos_ml[0])
db_neg_ml = np.array(db_neg_ml[0])
#db_pos_buscape = np.array(db_pos_buscape[0])

In [19]:
liwc = pd.read_csv(r'LIWC-original.txt',header=None)

palavras = np.array(liwc[0])
sentimentos = np.array(liwc[1])

pos = []
neg = []

for i in range(len(sentimentos)):
    if(sentimentos[i]==1):
        pos.append(palavras[i])
    else:
        neg.append(palavras[i])


        
#print(len(pos))
#print(pos)

#print(len(neg))

In [20]:
from nltk import sent_tokenize, word_tokenize, WordNetLemmatizer
 
 
lemmatizer = WordNetLemmatizer()
 
def check_pol(text):
    
    #Return a sentiment polarity: 0 = negative, 1 = positive

    pol = 0.0

    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        words = word_tokenize(raw_sentence)
        
        for word in words:
            negation = False
            if u'não' in text:
                negation = True
            if(word in pos):
                pol += 1
                #print('pos: ' + word)
            if(word in neg):
                #print('neg: ' + word)
                pol += -1
            if negation:
                pol = -1 * pol
   
    # sum greater than 0 => positive sentiment
    if pol >= 0:
        #return 'positive'
        return 1
    # negative sentiment
    #return 'negative'
    return 0

In [24]:
classificando_ml_positiva = [check_pol(text) for text in db_pos_ml]
classificando_ml_negativa = [check_pol(text) for text in db_neg_ml]

In [47]:
db_full_ml = np.append(db_pos_ml, db_neg_ml)
df_full_ml = DataFrame(db_full_ml)
classificando_ml_full = np.append(classificando_ml_positiva, classificando_ml_negativa)
df_classificando_ml_full = DataFrame(classificando_ml_full)

In [49]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

df_full_ml_2 = df_full_ml.apply(le.fit_transform)
df_full_ml_2.head()

#df_pos_buscape_2 = df_pos_buscape.apply(le.fit_transform)
#df_buscape_2.head()

,0
0,22415
1,43196
2,23705
3,25387
4,16391


In [50]:
# etapa 2 do processo para transformar o texto(variável categórica) em variável dummy para poder utilizar o SVM
# criando um objeto OneHotEncoder, e fazendo fit em todo df_2

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(df_full_ml_2)
#enc.fit(df_pos_buscape_2)

# 3. Transform
#onehotlabels_ml = enc.transform(df_pos_ml_2).toarray()
#onehotlabels_ml = enc.transform(df_pos_buscape_2).toarray()

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [51]:
from sklearn import svm

# Treinando o svm com kernel linear nos dados do arquivo ML.pos

clf = svm.SVC(kernel='linear', C=1).fit(df_full_ml_2, df_classificando_ml_full)


C:\Users\Arlon\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
# Testando o svm com os dados do Buscape.pos
y_pred = clf.predict(df_full_ml_2)
#clf.score(df_pos_buscape_2, classificando_buscape)

In [68]:
from sklearn.metrics import f1_score
#f1_score(y_true, y_pred, average='macro')
f1_score(df_classificando_ml_full, y_pred, average='macro')

C:\Users\Arlon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.43040105193951345